In [ ]:
# Original code from https://github.com/yhzhao343/brainbraille_decode
# Author: Yuhui Zhao

In [2]:
# try:
#     # Dirty trick to format the file nicely
#     from google.colab import drive

#     !pip install black[jupyter] --quiet

#     drive.mount("/content/drive")
#     !black /content/drive/MyDrive/brainbraille_new_pipeline.ipynb
# except:
#     pass
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    !pip install git+https://github.com/yhzhao343/brainbraille_decode.git
    !pip install onedrivedownloader

In [4]:
import os
import time
import pytz
from datetime import datetime
import msgpack
import msgpack_numpy as m
import numpy as np
from functools import partial
from copy import deepcopy
import fastFMRI

from fastFMRI.file_helpers import load_file, write_file
import warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np
from scipy.signal import butter, sosfilt
from sklearn.base import BaseEstimator, TransformerMixin
from fastFMRI.roi import (
    get_roi_from_flatten_t,
    get_calib_roi_from_flatten_roi,
    get_aggregated_roi_from_flatten,
)

from fastFMRI.roi import extract_from_4d_data_path_using_3d_mask
from joblib import Parallel, delayed
print("NOTE! it takes FOREVER to import brainbraille_decode for the first time")
import brainbraille_decode

from brainbraille_decode.lm import (
    grammar_info_gen,
    letter_label,
    forward_decode,
    viterbi_decode,
    add_k_smoothing_1d,
    add_k_smoothing_2d,
    viterbi_decode_with_grammar_from_hidden_state_proba,
    forward_decode_from_letter_proba_for_all_runs,
    viterbi_decode_from_letter_proba_for_all_runs,
    viterbi_decode_with_grammar_for_all_runs,
    hidden_state_proba_to_emission_proba,
    get_log_symbol_out_emission,
    viterbi_decode,
)
from brainbraille_decode.preprocessing import (
    ROIandCalibrationExtractor,
    ButterworthBandpassFilter,
    ZNormalizeByGroup,
    LeadingTrailingDataSlice,
    # DataSlice,
)
from brainbraille_decode.viterbi_decoder_helpers import (
    get_symbol_node_trans,
    letter_label_to_state_label,
    clf_fit_pred_score,
    param_result_format,
    tune_clf_results_pretty_print,
    get_slices_and_extract_data,
    tune_clf,
    clf_fit,
    clf_predict_proba,
    state_proba_to_letter_proba,
    lipo_param_search,
    tune_decode_smoothing,
    clf_fit_each_r,
    clf_predict_proba_each_r,
    letter_bigram_viterbi_with_grammar_decode,
    resolve_null,
    symbol_info_preprocess,
    joint_proba_of_hidden_state_sequence,
)
from brainbraille_decode.viterbi_decoder import StateProbaToLetterProb
from sklearn.model_selection import (
    LeaveOneOut,
    LeaveOneGroupOut,
    LeavePOut,
)
from brainbraille_decode.cross_validation import BrainBrailleCVGen
from lipo import GlobalOptimizer
from scipy.special import softmax
from numba import jit, prange, f8, u4, i4, i8, b1
import numba as nb

m.patch()
import matplotlib.pyplot as plt

plt.style.use("seaborn-v0_8")

NOTE! it takes FOREVER to import brainbraille_decode for the first time


ImportError: dlopen(/opt/homebrew/anaconda3/envs/brainbrailleenv/lib/python3.11/site-packages/_dlib_pybind11.cpython-311-darwin.so, 0x0002): tried: '/opt/homebrew/anaconda3/envs/brainbrailleenv/lib/python3.11/site-packages/_dlib_pybind11.cpython-311-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e' or 'arm64')), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/anaconda3/envs/brainbrailleenv/lib/python3.11/site-packages/_dlib_pybind11.cpython-311-darwin.so' (no such file), '/opt/homebrew/anaconda3/envs/brainbrailleenv/lib/python3.11/site-packages/_dlib_pybind11.cpython-311-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e' or 'arm64'))

In [23]:
# FILE_TO_USE = "18S"
# FILE_TO_USE = "6S"
FILE_TO_USE = "3S"
#FILE_TO_USE = '1S5_THAD'
# FILE_TO_USE = '1S5_FEYI'
info_dict = {
    "18S": {
        "ENV_VAR": "BRAINBRAILLE_INTERMEDIATE_18S_PER_RUN_RAW_DATA_FILE_PATH",
        "file_name": "brainbraille_intermediate_12s+6s_run_raw_data.bin",
        "url": "https://gtvault-my.sharepoint.com/:u:/g/personal/yzhao343_gatech_edu/EUK-mA0bnThOutq9_eOsJMIB0GUUn3DS7Tc8B-1UKRo7pA?e=HHrU5O",
    },
    "6S": {
        "ENV_VAR": "BRAINBRAILLE_INTERMEDIATE_6S_PER_RUN_RAW_DATA_FILE_PATH",
        "file_name": "brainbraille_intermediate_6s_run_raw_data.bin",
        "url": "https://gtvault-my.sharepoint.com/:u:/g/personal/yzhao343_gatech_edu/EW56YHomjM1Egsqh43pncpEBK5iTG2dI0ccBDX9gZp07wQ?e=XzLuNN",
    },
    "3S": {
        "ENV_VAR": "BRAINBRAILLE_INTERMEDIATE_3S_PER_RUN_RAW_DATA_FILE_PATH",
        "file_name": "brainbraille_intermediate_3s_per_run_raw_data.bin",
        "url": "https://gtvault-my.sharepoint.com/:u:/g/personal/yzhao343_gatech_edu/EYfSrpjQzcxKtVXkP1OeZrQBD6N1gNbiA3yqGjM6OhRoGg?e=Obj5Ho",
    },
    "1S5_THAD": {
        "ENV_VAR": "BRAINBRAILLE_INTERMEDIATE_1S5_THAD_PER_RUN_RAW_DATA_FILE_PATH",
        "file_name": "brainbraille_intermediate_thad_1s5_run_raw_data.bin",
        "url": "https://gtvault-my.sharepoint.com/:u:/g/personal/yzhao343_gatech_edu/EdR7OAnM9W5JjD2FcSxWYa4Bs9vtvU157Zug4QiKmHvTvQ?e=yUfaxv",
    },
    "1S5_FEYI": {
        "ENV_VAR": "BRAINBRAILLE_INTERMEDIATE_1S5_FEYI_PER_RUN_RAW_DATA_FILE_PATH",
        "file_name": "brainbraille_intermediate_feyi_1s5_run_raw_data.bin",
        "url": "https://gtvault-my.sharepoint.com/:u:/g/personal/yzhao343_gatech_edu/EUMLUnsFKNNNhkxWtf6q5pkBvqDWYHM817GuBSRywYJZ8A?e=8jodqH"
    },
}

ENV_VAR = info_dict[FILE_TO_USE]["ENV_VAR"]
file_name = info_dict[FILE_TO_USE]["file_name"]
url = info_dict[FILE_TO_USE]["url"]

if ENV_VAR in os.environ:
    brainbraille_intermediate_per_run_raw_data_file_path = os.environ[ENV_VAR]
else:
    brainbraille_intermediate_per_run_raw_data_file_path = file_name
    if not os.path.exists(brainbraille_intermediate_per_run_raw_data_file_path):
        from onedrivedownloader import download

        # If the link expires, let me know and I will renew the link
        download(
            url=url,
            filename=brainbraille_intermediate_per_run_raw_data_file_path,
        )
brainbraille_per_run_data = msgpack.unpackb(
    load_file(brainbraille_intermediate_per_run_raw_data_file_path, "rb"),
    strict_map_key=False,
)
per_run_data = brainbraille_per_run_data["per_run_data"]
# grammar_info = brainbraille_per_run_data["grammar_info"]
LETTERS_TO_DOT = brainbraille_per_run_data["LETTERS_TO_DOT"]
if ' ' not in LETTERS_TO_DOT:
    LETTERS_TO_DOT[' '] = {'b': 0, 'f_l': 0, 'f_r': 0, 'h_l': 0, 'h_r': 0, 't': 0}

100%|█████████████████████████████████████| 2.08G/2.08G [11:18<00:00, 3.06MiB/s]


In [16]:
RUN_SVM = True

if FILE_TO_USE == "18S":
    TRAILLING_TIME_S = 6
    LEADING_TIME_S = 0
    DELAY_S = 12
else:
    TRAILLING_TIME_S = 6
    LEADING_TIME_S = 6
    DELAY_S = 3

SVM_n_calls = 256
RUN_BANDPASS = True           #@param {type:"boolean"}
if RUN_BANDPASS:
    BANDPASS_LOW_CUT = 0.01      #@param {type:"slider", min:0.000005, max:0.1, step:0.000005}
    BANDPASS_HIGH_CUT = 0.2       #@param {type:"slider", min:0.11, max:1, step:0.005}
    BANDPASS_ORDER = 1            #@param {type:"slider", "min":1, max:6, step:1}
Z_NORM = True                 #@param {type:"boolean"}
# EXTRA_TIME_S = 3
TR_s = per_run_data[0]["TR_s"]
STIMULI_LABEL_INTERVAL_s = per_run_data[0]["EVENT_INTERVAL_S"]
# EXTRA_FRAME = int(EXTRA_TIME_S / TR_s)
DELAY_FRAME = int(DELAY_S / TR_s)
SF_Hz = 1 / TR_s
NUM_FRAME_PER_LABEL = int(STIMULI_LABEL_INTERVAL_s / TR_s)
EVENT_LEN_S = per_run_data[0]["EVENT_LEN_S"]
EVENT_INTERVAL_S = per_run_data[0]["EVENT_INTERVAL_S"]
EVENT_LEN_FRAME = int(EVENT_LEN_S / TR_s)
EVENT_INTERVAL_FRAME = int(EVENT_INTERVAL_S / TR_s)
regressor_types = per_run_data[0]["regressor_types"]
inner_n_jobs = -1
letter_labels = [d_i["letter_label"] for d_i in per_run_data]
grammar_info = grammar_info_gen(letter_labels, EVENT_LEN_S, False)
LETTERS_TO_DOT_array = np.array(
    [[LETTERS_TO_DOT[l][r] for r in regressor_types] for l in letter_label], dtype=bool
)

TRAILLING_FRAME = int(TRAILLING_TIME_S / TR_s)
LEADING_FRAME = int(LEADING_TIME_S / TR_s)

Using cached response because an exception occurred:
[Errno 2] No such file or directory: 'None/HParse'


In [ ]:
print(letter_labels)
print(f"TR_s: {TR_s}s\nSTIMULI_LABEL_INTERVAL_s: {STIMULI_LABEL_INTERVAL_s}s\nEVENT_LEN_S: {EVENT_LEN_S}s\nEVENT_INTERVAL_S: {EVENT_INTERVAL_S}s\nEVENT_LEN_FRAME: {EVENT_LEN_FRAME}\nEVENT_INTERVAL_FRAME: {EVENT_INTERVAL_FRAME}")

In [14]:
subs = np.array([info["sub"] for info in per_run_data], dtype=int)
runs = np.array([info["run"] for info in per_run_data], dtype=int)
sess = np.array([info["ses"] for info in per_run_data], dtype=int)
indx = np.arange(subs.size)
print(f"indx: {np.array2string(indx, max_line_width=120)}")
print(f"subs: {np.array2string(subs, max_line_width=120)}")
print(f"runs: {np.array2string(runs, max_line_width=120)}")
print(f"sess: {np.array2string(sess, max_line_width=120)}")



indx: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37]
subs: [1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 4 4 4 4]
runs: [1 2 3 4 5 1 2 3 4 5 6 1 2 3 4 5 6 1 2 3 4 5 6 7 8 1 2 3 4 5 6 7 1 2 1 2 3 4]
sess: [1 1 1 1 1 2 2 2 2 2 2 1 1 1 1 1 1 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 1 1 1 1 1 1]


In [15]:
VERBOSE = True
Z_NORM = True
sub = 1
# sub = 2
n = 1
test_sub = 1
cv_sub_sampe_size = np.iinfo(np.int32).max
# cv_sub_sampe_size = 5000
sub_mask = (subs == 1) | (subs == 2) | (subs == 4)
# sub_mask = (subs == 1) | (subs == 3)
selected_ind = indx[sub_mask]
selected_sub = subs[sub_mask]
selected_run = runs[sub_mask]
selected_ses = sess[sub_mask]

print(f"indx: {np.array2string(selected_ind, max_line_width=120)}")
print(f"subs: {np.array2string(selected_sub, max_line_width=120)}")
print(f"runs: {np.array2string(selected_run, max_line_width=120)}")
print(f"sess: {np.array2string(selected_ses, max_line_width=120)}")
cv_generator = BrainBrailleCVGen(selected_sub, selected_run, selected_ses, selected_ind)

per_run_state_label = letter_label_to_state_label(
    [x_i["letter_label"] for x_i in [per_run_data[i] for i in selected_ind]],
    LETTERS_TO_DOT,
    regressor_types,
)
for i, state_label in zip(selected_ind, per_run_state_label):
    per_run_data[i]["state_label"] = state_label

EXPERIMENT_TYPE="subj_dependent"
# EXPERIMENT_TYPE = "subj_independent"
# EXPERIMENT_TYPE = "subj_adaptive"
# EXPERIMENT_TYPE == "subj_independent_LORO"
# EXPERIMENT_TYPE == "subj_adaptive_LORO"

if EXPERIMENT_TYPE == "subj_dependent":
    # # Subject dependent leave n run out:
    train_test_split = cv_generator.sub_dependent_leave_n_run_out(sub, n)
    train_vali_spliter = LeaveOneOut()
elif EXPERIMENT_TYPE == "subj_independent_LORO":
    train_test_split = cv_generator.sub_independent_leave_one_sub_out_test_sub(test_sub)
    train_vali_spliter = LeaveOneOut()
elif EXPERIMENT_TYPE == "subj_adaptive_LORO":
    train_test_split = cv_generator.sub_adaptive_leave_one_sub_out_test_sub(test_sub)
    train_vali_spliter = LeaveOneOut()
elif EXPERIMENT_TYPE == "subj_independent":
    # # Subject independent:
    train_test_split = cv_generator.sub_independent_leave_one_sub_out_test_sub(test_sub)
    train_train_i_list_all_fold, train_valid_i_list_all_fold = cv_generator.sub_independent_leave_one_sub_out_test_sub_train_valid(test_sub)
    if "train_vali_spliter" in globals():
        del train_vali_spliter
elif EXPERIMENT_TYPE == "subj_adaptive":
    # # Subject adaptive:
    train_test_split = cv_generator.sub_adaptive_leave_one_sub_out_test_sub(test_sub)
    train_train_i_list_all_fold, train_valid_i_list_all_fold = cv_generator.sub_adaptive_leave_one_sub_out_test_sub_train_valid(test_sub)
    if "train_vali_spliter" in globals():
        del train_vali_spliter

fit_roi_all_sub=False
if type(train_test_split) is tuple:
    for fold_i, (train_i, test_i) in enumerate(zip(*train_test_split)):
        print(f"fold: {fold_i}\n\ttrain ({len(train_i):02}): {train_i}\n\ttest  ({len(test_i):02}): {test_i}")
        train_i_sub_set = set(subs[train_i])
        test_i_sub_set = set(subs[test_i])
        if not fit_roi_all_sub :
            if (len(test_i_sub_set - train_i_sub_set) > 0):
                fit_roi_all_sub = True

if "train_train_i_list_all_fold" in globals():
    print("=== example train valid  ===")
    for train_train_i, train_valid_i in zip(train_train_i_list_all_fold[0], train_valid_i_list_all_fold[0]):
        print(train_train_i)
        print(train_valid_i)

indx: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 34 35 36 37]
subs: [1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 4 4 4 4]
runs: [1 2 3 4 5 1 2 3 4 5 6 1 2 3 4 5 6 1 2 3 4 5 6 7 8 1 2 3 4 5 6 7 1 2 3 4]
sess: [1 1 1 1 1 2 2 2 2 2 2 1 1 1 1 1 1 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 1 1 1 1]
fold: 0
	train (10): [ 1  2  3  4  5  6  7  8  9 10]
	test  (01): [0]
fold: 1
	train (10): [ 0  2  3  4  5  6  7  8  9 10]
	test  (01): [1]
fold: 2
	train (10): [ 0  1  3  4  5  6  7  8  9 10]
	test  (01): [2]
fold: 3
	train (10): [ 0  1  2  4  5  6  7  8  9 10]
	test  (01): [3]
fold: 4
	train (10): [ 0  1  2  3  5  6  7  8  9 10]
	test  (01): [4]
fold: 5
	train (10): [ 0  1  2  3  4  6  7  8  9 10]
	test  (01): [5]
fold: 6
	train (10): [ 0  1  2  3  4  5  7  8  9 10]
	test  (01): [6]
fold: 7
	train (10): [ 0  1  2  3  4  5  6  8  9 10]
	test  (01): [7]
fold: 8
	train (10): [ 0  1  2  3  4  5  6  7  9 10]
	test  (01): [8]
fold: 9
	train

In [ ]:
if RUN_SVM:
    test_letter_label_list = []
    test_state_label_list = []

    test_naive_prob_letter_pred_list = []

    region_train_valid_state_label_list = []
    region_train_valid_state_pred_list = []
    region_test_state_pred_list = []

    region_cv_score_train_list = []
    region_cv_score_test_list = []

    test_naive_prob_letter_test_acc_list = []

    stimulus_forward_decode_letter_list = []
    stimulus_forward_decode_train_acc_list = []
    stimulus_forward_decode_test_acc_list = []

    stimulus_viterbi_decode_letter_list = []
    stimulus_viterbi_decode_train_acc_list = []
    stimulus_viterbi_decode_test_acc_list = []

    mackenzie_soukoreff_forward_decode_letter_list = []
    mackenzie_soukoreff_forward_decode_train_acc_list = []
    mackenzie_soukoreff_forward_decode_test_acc_list = []

    mackenzie_soukoreff_viterbi_decode_letter_list = []
    mackenzie_soukoreff_viterbi_decode_train_acc_list = []
    mackenzie_soukoreff_viterbi_decode_test_acc_list = []

    stimulus_pred_letter_viterbi_decode_letter_label_list = []
    stimulus_pred_letter_viterbi_decode_train_acc_list = []
    stimulus_pred_letter_viterbi_decode_test_acc_list = []

    aw2aw_stimulus_pred_letter_viterbi_decode_letter_label_list = []
    aw2aw_stimulus_pred_letter_viterbi_decode_train_acc_list = []
    aw2aw_stimulus_pred_letter_viterbi_decode_test_acc_list = []

    mackenzie_soukoreff_pred_letter_viterbi_decode_letter_label_list = []
    mackenzie_soukoreff_pred_letter_viterbi_decode_train_list = []
    mackenzie_soukoreff_pred_letter_viterbi_decode_test_list = []

    aw2aw_mackenzie_soukoreff_pred_letter_viterbi_decode_letter_label_list = []
    aw2aw_mackenzie_soukoreff_pred_letter_viterbi_decode_train_list = []
    aw2aw_mackenzie_soukoreff_pred_letter_viterbi_decode_test_list = []

    total_time_start_i = time.time()
    if "train_test_spliter" in globals():
        if type(train_test_spliter) is LeaveOneOut:
            train_test_split = partial(train_test_spliter.split, X=selected_run)
            train_test_iter = train_test_split()
    elif type(train_test_split) is tuple:
        train_test_iter = zip(*train_test_split)

    for fold_i, (train_i, test_i) in enumerate(train_test_iter):
        fold_i_start_time = time.time()

        num_letter_label = len(per_run_data[indx[train_i[0]]]["letter_label"])

        roi_and_calib_extractor = ROIandCalibrationExtractor(fit_roi_all_sub=fit_roi_all_sub)

        data_slicer = LeadingTrailingDataSlice(
            leading_frame = LEADING_FRAME,
            event_len_frame = EVENT_LEN_FRAME,
            trailling_frame = TRAILLING_FRAME,
            delay_frame = DELAY_FRAME,
            event_interval_frame = EVENT_INTERVAL_FRAME,
            fill_moving_avg_len_frame = EVENT_LEN_FRAME,
            num_slices=num_letter_label,
            feature_mask=None,
        )

        steps = [("roi_and_calib_extractor", roi_and_calib_extractor)]
        if RUN_BANDPASS:
            butter_filter = ButterworthBandpassFilter(
                BANDPASS_LOW_CUT,
                BANDPASS_HIGH_CUT,
                SF_Hz,
                BANDPASS_ORDER,
            )
            steps.append(("bandpass", butter_filter))
        steps.append(("sliding_window", data_slicer))
        if Z_NORM:
            steps.append(("z_norm", ZNormalizeByGroup()))

        roi_extract_and_filter = Pipeline(steps)

        if "train_vali_spliter" in globals():
            if (type(train_vali_spliter) is LeaveOneOut):
                train_vali_split = partial(train_vali_spliter.split, X=train_i)
                train_train_i_list, train_valid_i_list = zip(*train_vali_split())
        else:
            train_train_i_list = train_train_i_list_all_fold[fold_i]
            train_valid_i_list = train_valid_i_list_all_fold[fold_i]
            # train_vali_split
        # break
        (
            train_letter_label_i,
            test_letter_label_i,
            train_state_label_i,
            test_state_label_i,
            train_train_extracted_flatten_Xs,
            train_train_state_flatten_labels,
            train_train_letter_flatten_labels,
            train_train_state_flatten_labels_by_r,
            train_valid_extracted_flatten_Xs,
            train_valid_state_flatten_labels,
            train_valid_letter_flatten_labels,
            train_valid_state_flatten_labels_by_r,
            test_sub_i,
            train_sub_i,
            train_state_flatten_label_i,
            train_data_i,
            test_data_i,
        ) = get_slices_and_extract_data(
            indx,
            fold_i,
            per_run_data,
            subs,
            train_i,
            test_i,
            train_train_i_list,
            train_valid_i_list,
            roi_extract_and_filter,
            Z_NORM,
            n_jobs = -1,
        )
        # test_naive_prob_letter_label_list += test_letter_label_i
        test_letter_label_i_flatten = np.concatenate(test_letter_label_i)
        test_letter_label_list += test_letter_label_i
        test_state_label_list += test_state_label_i
        train_valid_label_flatten = np.concatenate(train_valid_letter_flatten_labels)

        train_valid_state_flatten_labels_by_r = [[np.array(run_i).astype(np.bool_) for run_i in fold_i] for fold_i in train_valid_state_flatten_labels_by_r]
        train_valid_state_flatten_labels = [[np.array(run_i).astype(np.bool_) for run_i in fold_i] for fold_i in train_valid_state_flatten_labels]
        train_train_state_flatten_labels_by_r = [[np.array(run_i).astype(np.bool_) for run_i in fold_i] for fold_i in train_train_state_flatten_labels_by_r]
        train_train_state_flatten_labels = [[np.array(run_i).astype(np.bool_) for run_i in fold_i] for fold_i in train_train_state_flatten_labels]


        clf = SVC(kernel="rbf", cache_size=2000)
        param_category_keys = []
        svc_param = {
            "C": [1.0, 1000.0],
            "gamma": [0.001, 0.1],
        }
        log_args_keys = ["C", "gamma"]
        flexible_bounds = {"C": [False, True], "gamma": [False, True]}

        # n_calls = 256
        cv_sub_sampe_ratio = min(
            cv_sub_sampe_size / len(train_train_extracted_flatten_Xs[0]), 1.0
        )
        region_cv_clfs, region_cv_tune_clf_results, region_cv_tune_clf_history = zip(
            *Parallel(n_jobs=-1)(
                delayed(tune_clf)(
                    train_train_extracted_flatten_Xs,
                    train_train_state_flatten_labels_by_r[r_i],
                    train_valid_extracted_flatten_Xs,
                    train_valid_state_flatten_labels_by_r[r_i],
                    deepcopy(clf),
                    svc_param,
                    param_category_keys,
                    log_args_keys,
                    flexible_bounds,
                    optimum_val_max=1.0,
                    maximize=True,
                    flexible_bound_threshold=0.1,
                    random_state=42,
                    n_calls=SVM_n_calls,
                    sub_sample_ratio=cv_sub_sampe_ratio,
                )
                for r_i in range(len(regressor_types))
            )
        )
        if VERBOSE:
            tune_clf_results_pretty_print(region_cv_tune_clf_results, regressor_types)
            fold_i_clf_tune_finish_time = time.time()
            print(
                f"clf tune time: {fold_i_clf_tune_finish_time - fold_i_start_time:.3f}"
            )
        region_cv_params, region_cv_score, _ = zip(*region_cv_tune_clf_results)
        region_cv_score_train_list.append(region_cv_score)
        # clf.set_params(probability=True)
        region_clfs = [
            deepcopy(clf).set_params(**param, probability=True)
            for r, param in zip(regressor_types, region_cv_params)
        ]
        if Z_NORM:
            roi_extract_and_filter.set_params(
                z_norm__test_group=test_sub_i, z_norm__train_group=train_sub_i
            )
        train_data_extracted_i = roi_extract_and_filter.fit_transform(train_data_i)
        test_data_extracted_i = roi_extract_and_filter.transform(test_data_i)

        num_train_run, num_train_trial, num_train_frame, num_train_roi = (
            train_data_extracted_i.shape
        )
        num_test_run, num_test_trial, num_test_frame, num_test_roi = (
            test_data_extracted_i.shape
        )

        train_data_flatten_i = train_data_extracted_i.reshape(
            np.prod(train_data_extracted_i.shape[:2]),
            np.prod(train_data_extracted_i.shape[2:]),
        )
        test_data_flatten_i = test_data_extracted_i.reshape(
            np.prod(test_data_extracted_i.shape[:2]),
            np.prod(test_data_extracted_i.shape[2:]),
        )

        region_clfs = Parallel(n_jobs=-1)(
            delayed(clf_fit)(
                clf,
                train_data_flatten_i,
                [run_i[r_i] for run_i in train_state_flatten_label_i],
            )
            for r_i, clf in enumerate(region_clfs)
        )
        clf_train_end_time = time.time()
        print(f"clf train time: {clf_train_end_time - fold_i_clf_tune_finish_time:.3f}")

        test_predict_state = np.array(
            Parallel(n_jobs=-1)(
                delayed(clf_predict_proba)(clf, test_data_flatten_i)
                for r_i, clf in enumerate(region_clfs)
            )
        )
        test_predict_state = test_predict_state[:, :, 1].squeeze().T
        test_predict_state_per_run = test_predict_state.reshape(
            (num_test_run, num_test_trial, num_test_roi)
        )
        clf_inference_end_time = time.time()
        print(f"clf inference time: {clf_inference_end_time - clf_train_end_time:.3f}")

        (
            test_naive_letter_proba_marg_per_run,
            test_naive_letter_proba_ind_per_run,
        ) = zip(
            *[
                state_proba_to_letter_proba(
                    np.ascontiguousarray(run_i), LETTERS_TO_DOT_array
                )
                for run_i in test_predict_state_per_run
            ]
        )
        test_naive_letter_prob_letter_per_run = [
            [letter_label[l_i] for l_i in run_i]
            for run_i in test_naive_letter_proba_ind_per_run
        ]
        region_test_state_pred_list.append(test_predict_state_per_run > 0.5)

        test_naive_prob_letter_pred_list += test_naive_letter_prob_letter_per_run
        # print(test_letter_label_i_flatten)
        # print(np.concatenate(test_naive_letter_prob_letter_per_run))
        test_naive_prob_letter_test_accuracy = accuracy_score(
            test_letter_label_i_flatten,
            np.concatenate(test_naive_letter_prob_letter_per_run),
        )
        test_naive_prob_letter_test_acc_list.append(test_naive_prob_letter_test_accuracy)

        clf_train_inf_on_cv_split_start_time = time.time()
        region_clfs_per_train_valid_split = Parallel(n_jobs=-1)(
            delayed(clf_fit_each_r)(region_clfs, train_train_x, train_train_yr)
            for train_train_x, train_train_yr in zip(
                train_train_extracted_flatten_Xs, train_train_state_flatten_labels
            )
        )

        train_valid_state_per_run = Parallel(n_jobs=-1)(
            delayed(clf_predict_proba_each_r)(clfs, x_i)
            for clfs, x_i in zip(
                region_clfs_per_train_valid_split, train_valid_extracted_flatten_Xs
            )
        )
        train_valid_state_per_run = [x_i[:, :, 1].T for x_i in train_valid_state_per_run]

        (
            train_valid_letter_proba_marg_per_run,
            train_valid_letter_proba_ind_per_run,
        ) = zip(
            *[
                state_proba_to_letter_proba(np.ascontiguousarray(run_i), LETTERS_TO_DOT_array)
                for run_i in train_valid_state_per_run
            ]
        )
        train_valid_letter_prob_letter_per_run = np.concatenate([
            [letter_label[l_i] for l_i in run_i]
            for run_i in train_valid_letter_proba_ind_per_run
        ])
        clf_train_inf_on_cv_split_end_time = time.time()
        naive_prob_letter_train_accuracy = accuracy_score(
            train_valid_label_flatten,
            train_valid_letter_prob_letter_per_run,
        )

        train_valid_state_per_run = [run_i > 0.5 for run_i in train_valid_state_per_run]


        test_acc_per_r = [
            accuracy_score(
                np.concatenate([[l_i[i] for l_i in fold_i] for fold_i in train_valid_state_flatten_labels]),
                np.concatenate([fold_i[:, i] for fold_i in train_valid_state_per_run]),
            )
            for i in range(len(regressor_types))
        ]
        region_train_valid_state_label_list.append(train_valid_state_per_run)
        region_train_valid_state_pred_list.append(train_valid_state_flatten_labels)

        print("----- region train valid results -----")
        for i, r in enumerate(regressor_types):
            print(f"region:{r:>4} acc: {test_acc_per_r[i]:7.4f}")
        print("----- ------------------- ----")
        region_cv_score_test_list.append(test_acc_per_r)

        print(
            f"Naive prob letter accuracy: train:{naive_prob_letter_train_accuracy:.4f} test:{test_naive_prob_letter_test_accuracy:.4f}"
        )
        print(
            f"clf train-inference on cv split time: {clf_train_inf_on_cv_split_end_time - clf_train_inf_on_cv_split_start_time:.3f}"
        )
        if FILE_TO_USE in ["6S", "18S"]:
            continue

        initial_proba = np.zeros(len(letter_label), dtype=np.float64)
        initial_proba[0] = 1.0
        param_category_keys = []
        log_args_keys = ["uni_k", "bi_k"]
        # log_args_keys = []
        decode_smoothing_param = {
            "uni_k": [1e-3, 1e3],
            "bi_k": [1e-3, 1e3],
            "ip_k": [1e-3, 1e3],
        }
        flexible_bounds = {
            "uni_k": [False, True],
            "bi_k": [False, True],
            "ip_k": [False, True],
        }
        n_calls = 1024
        smoothing_1d = add_k_smoothing_1d
        smoothing_2d = add_k_smoothing_2d

        corpus = "stimulus"
        print(f"Letter decode using corpus: {corpus}")
        uni_gram_count = np.array(
            grammar_info[f"{corpus}_letter_one_gram_count"], dtype=np.float64
        )
        bigram_gram_count = np.array(
            grammar_info[f"{corpus}_letter_bigram_count"], dtype=np.float64
        )

        (
            optimized_forward_decoder,
            optimized_forward_decoder_results,
            optimized_forward_decoder_history,
        ) = tune_decode_smoothing(
            train_valid_letter_proba_marg_per_run,
            train_valid_letter_flatten_labels,
            uni_gram_count,
            bigram_gram_count,
            initial_proba,
            letter_label,
            forward_decode_from_letter_proba_for_all_runs,
            smoothing_1d,
            smoothing_2d,
            decode_smoothing_param,
            param_category_keys,
            log_args_keys,
            flexible_bounds,
            n_calls=n_calls,
        )

        test_forward_decode_letter_per_run = optimized_forward_decoder(
            letter_proba_per_run=test_naive_letter_proba_marg_per_run
        )
        stimulus_forward_decode_letter_list += test_forward_decode_letter_per_run
        stim_forward_letter_accuracy = accuracy_score(
            test_letter_label_i_flatten,
            np.concatenate(test_forward_decode_letter_per_run),
        )
        stim_forward_decode_train_acc = optimized_forward_decoder_results[1]
        stimulus_forward_decode_train_acc_list.append(stim_forward_decode_train_acc)
        stimulus_forward_decode_test_acc_list.append(stim_forward_letter_accuracy)

        print(
            f"\tForward decode: train:{stim_forward_decode_train_acc:.4f} test:{stim_forward_letter_accuracy:.4f} {param_result_format(optimized_forward_decoder_results[0])}"
        )

        (
            optimized_viterbi_decoder,
            optimized_viterbi_decoder_results,
            optimized_viterbi_decoder_history,
        ) = tune_decode_smoothing(
            train_valid_letter_proba_marg_per_run,
            train_valid_letter_flatten_labels,
            uni_gram_count,
            bigram_gram_count,
            initial_proba,
            letter_label,
            viterbi_decode_from_letter_proba_for_all_runs,
            smoothing_1d,
            smoothing_2d,
            decode_smoothing_param,
            param_category_keys,
            log_args_keys,
            flexible_bounds,
            n_calls=n_calls,
        )

        test_viterbi_decode_letter_per_run = optimized_viterbi_decoder(
            letter_proba_per_run=test_naive_letter_proba_marg_per_run
        )
        stimulus_viterbi_decode_letter_list += test_viterbi_decode_letter_per_run
        stim_viterbi_letter_accuracy = accuracy_score(
            test_letter_label_i_flatten,
            np.concatenate(test_viterbi_decode_letter_per_run),
        )
        stim_viterbi_decode_train_acc = optimized_viterbi_decoder_results[1]
        stimulus_viterbi_decode_train_acc_list.append(stim_viterbi_decode_train_acc)
        stimulus_viterbi_decode_test_acc_list.append(stim_viterbi_letter_accuracy)
        print(
            f"\tViterbi decode: train:{stim_viterbi_decode_train_acc:.4f} test:{stim_viterbi_letter_accuracy:.4f} {param_result_format(optimized_viterbi_decoder_results[0])}"
        )

        corpus = "mackenzie_soukoreff"
        print(f"Letter decode using corpus: {corpus}")
        uni_gram_count = np.array(
            grammar_info[f"{corpus}_letter_one_gram_count"], dtype=np.float64
        )
        bigram_gram_count = np.array(
            grammar_info[f"{corpus}_letter_bigram_count"], dtype=np.float64
        )

        (
            optimized_forward_decoder,
            optimized_forward_decoder_results,
            optimized_forward_decoder_history,
        ) = tune_decode_smoothing(
            train_valid_letter_proba_marg_per_run,
            train_valid_letter_flatten_labels,
            uni_gram_count,
            bigram_gram_count,
            initial_proba,
            letter_label,
            forward_decode_from_letter_proba_for_all_runs,
            smoothing_1d,
            smoothing_2d,
            decode_smoothing_param,
            param_category_keys,
            log_args_keys,
            flexible_bounds,
            n_calls=n_calls,
        )

        test_forward_decode_letter_per_run = optimized_forward_decoder(
            letter_proba_per_run=test_naive_letter_proba_marg_per_run
        )
        mackenzie_soukoreff_forward_decode_letter_list += (
            test_forward_decode_letter_per_run
        )
        mackenzie_soukoreff_forward_letter_accuracy = accuracy_score(
            test_letter_label_i_flatten,
            np.concatenate(test_forward_decode_letter_per_run),
        )
        mackenzie_soukoreff_forward_decode_train_acc = (
            optimized_forward_decoder_results[1]
        )
        mackenzie_soukoreff_forward_decode_train_acc_list.append(
            mackenzie_soukoreff_forward_decode_train_acc
        )
        mackenzie_soukoreff_forward_decode_test_acc_list.append(
            mackenzie_soukoreff_forward_letter_accuracy
        )
        print(
            f"\tForward decode: train:{mackenzie_soukoreff_forward_decode_train_acc:.4f} test:{mackenzie_soukoreff_forward_letter_accuracy:.4f} {param_result_format(optimized_forward_decoder_results[0])}"
        )

        (
            optimized_viterbi_decoder,
            optimized_viterbi_decoder_results,
            optimized_viterbi_decoder_history,
        ) = tune_decode_smoothing(
            train_valid_letter_proba_marg_per_run,
            train_valid_letter_flatten_labels,
            uni_gram_count,
            bigram_gram_count,
            initial_proba,
            letter_label,
            viterbi_decode_from_letter_proba_for_all_runs,
            smoothing_1d,
            smoothing_2d,
            decode_smoothing_param,
            param_category_keys,
            log_args_keys,
            flexible_bounds,
            n_calls=n_calls,
        )

        test_viterbi_decode_letter_per_run = optimized_viterbi_decoder(
            letter_proba_per_run=test_naive_letter_proba_marg_per_run
        )
        mackenzie_soukoreff_viterbi_decode_letter_list += (
            test_viterbi_decode_letter_per_run
        )
        mackenzie_soukoreff_viterbi_letter_accuracy = accuracy_score(
            test_letter_label_i_flatten,
            np.concatenate(test_viterbi_decode_letter_per_run),
        )
        mackenzie_soukoreff_viterbi_decode_train_acc = (
            optimized_viterbi_decoder_results[1]
        )
        mackenzie_soukoreff_viterbi_decode_train_acc_list.append(
            mackenzie_soukoreff_viterbi_decode_train_acc
        )
        mackenzie_soukoreff_viterbi_decode_test_acc_list.append(
            mackenzie_soukoreff_viterbi_letter_accuracy
        )

        print(
            f"\tViterbi decode: train:{mackenzie_soukoreff_viterbi_decode_train_acc:.4f} test:{mackenzie_soukoreff_viterbi_letter_accuracy:.4f} {param_result_format(optimized_viterbi_decoder_results[0])}"
        )

        grammar_info = grammar_info_gen(letter_labels, EVENT_LEN_S, False)
        corpus = "stimulus"
        print(f"Grammar decode using corpus: {corpus}")
        initial_proba = np.zeros(len(letter_label), dtype=np.float64)
        initial_proba[0] = 1.0
        param_category_keys = []
        log_args_keys = ["uni_k", "bi_k"]
        log_args_keys = []
        decode_smoothing_param = {
            "uni_k": [1e-3, 1e3],
            "bi_k": [1e-3, 1e3],
            # "ip_k":  [0.001/27, 1.0/27],
            # "insert_panelty": [-10, 10],
        }
        flexible_bounds = {
            "uni_k": [False, True],
            "bi_k": [False, True],
            # "ip_k": [False, True],
            # "insert_panelty": [True, True],
        }
        n_calls = 1024

        # ===== stimulus - SFFW =====
        prefix = ""
        symbol_nodes = grammar_info[f"{prefix}{corpus}_words_node_symbols"]
        symbol_node_edges = grammar_info[f"{prefix}{corpus}_words_link_start_end"]
        node_output_letter = grammar_info[f"unique_{corpus}_word_dictionary"]

        (
            symbol_node_trans,
            symbol_nodes_spelling_ndx,
            symbol_nodes_out_spelling_matrix,
            symbol_nodes_out_len,
            dummy_start_node_i,
            end_node_index,
        ) = symbol_info_preprocess(
            letter_label, symbol_nodes, symbol_node_edges, node_output_letter
        )

        viterbi_decode_with_grammar_for_all_runs_with_meta_data = partial(
            viterbi_decode_with_grammar_for_all_runs,
            dummy_start_node_i=dummy_start_node_i,
            end_node_index=end_node_index,
            symbol_node_trans=symbol_node_trans,
            symbol_nodes_spelling_ndx=symbol_nodes_spelling_ndx,
            symbol_nodes_out_spelling_matrix=symbol_nodes_out_spelling_matrix,
            symbol_nodes_out_len=symbol_nodes_out_len,
        )
        (
            optimized_viterbi_decode_with_grammar,
            optimized_viterbi_decode_with_grammar_results,
            optimized_viterbi_decode_with_grammar_history,
        ) = tune_decode_smoothing(
            train_valid_letter_proba_marg_per_run,
            train_valid_letter_flatten_labels,
            uni_gram_count,
            bigram_gram_count,
            initial_proba,
            letter_label,
            viterbi_decode_with_grammar_for_all_runs_with_meta_data,
            smoothing_1d,
            smoothing_2d,
            decode_smoothing_param,
            param_category_keys,
            log_args_keys,
            flexible_bounds,
            n_calls=n_calls,
        )

        test_stim_viterbi_decode_with_grammar_per_run = (
            optimized_viterbi_decode_with_grammar(
                letter_proba_per_run=test_naive_letter_proba_marg_per_run
            )
        )
        stimulus_pred_letter_viterbi_decode_letter_label_list += (
            test_stim_viterbi_decode_with_grammar_per_run
        )
        stim_viterbi_decode_with_grammar_letter_accuracy = accuracy_score(
            test_letter_label_i_flatten,
            np.concatenate(test_stim_viterbi_decode_with_grammar_per_run),
        )
        stimulus_pred_letter_viterbi_decode_train_acc = (
            optimized_viterbi_decode_with_grammar_results[1]
        )
        stimulus_pred_letter_viterbi_decode_train_acc_list.append(
            stimulus_pred_letter_viterbi_decode_train_acc
        )
        stimulus_pred_letter_viterbi_decode_test_acc_list.append(
            stim_viterbi_decode_with_grammar_letter_accuracy
        )
        print(
            f"\tSFFW  viterbi decode: train:{stimulus_pred_letter_viterbi_decode_train_acc:.4f} test:{stim_viterbi_decode_with_grammar_letter_accuracy:.4f} {param_result_format(optimized_viterbi_decode_with_grammar_results[0])}"
        )

        # ===== stimulus - aw2aw =====
        prefix = "aw2aw_"
        symbol_nodes = grammar_info[f"{prefix}{corpus}_words_node_symbols"]
        symbol_node_edges = grammar_info[f"{prefix}{corpus}_words_link_start_end"]
        node_output_letter = grammar_info[f"unique_{corpus}_word_dictionary"]

        (
            symbol_node_trans,
            symbol_nodes_spelling_ndx,
            symbol_nodes_out_spelling_matrix,
            symbol_nodes_out_len,
            dummy_start_node_i,
            end_node_index,
        ) = symbol_info_preprocess(
            letter_label, symbol_nodes, symbol_node_edges, node_output_letter
        )

        viterbi_decode_with_grammar_for_all_runs_with_meta_data = partial(
            viterbi_decode_with_grammar_for_all_runs,
            dummy_start_node_i=dummy_start_node_i,
            end_node_index=end_node_index,
            symbol_node_trans=symbol_node_trans,
            symbol_nodes_spelling_ndx=symbol_nodes_spelling_ndx,
            symbol_nodes_out_spelling_matrix=symbol_nodes_out_spelling_matrix,
            symbol_nodes_out_len=symbol_nodes_out_len,
        )
        (
            aw2aw_optimized_viterbi_decode_with_grammar,
            aw2aw_optimized_viterbi_decode_with_grammar_results,
            aw2aw_optimized_viterbi_decode_with_grammar_history,
        ) = tune_decode_smoothing(
            train_valid_letter_proba_marg_per_run,
            train_valid_letter_flatten_labels,
            uni_gram_count,
            bigram_gram_count,
            initial_proba,
            letter_label,
            viterbi_decode_with_grammar_for_all_runs_with_meta_data,
            smoothing_1d,
            smoothing_2d,
            decode_smoothing_param,
            param_category_keys,
            log_args_keys,
            flexible_bounds,
            n_calls=n_calls,
        )
        aw2aw_test_stim_viterbi_decode_with_grammar_per_run = (
            aw2aw_optimized_viterbi_decode_with_grammar(
                letter_proba_per_run=test_naive_letter_proba_marg_per_run
            )
        )
        aw2aw_stimulus_pred_letter_viterbi_decode_letter_label_list += (
            aw2aw_test_stim_viterbi_decode_with_grammar_per_run
        )
        aw2aw_stim_viterbi_decode_with_grammar_letter_accuracy = accuracy_score(
            test_letter_label_i_flatten,
            np.concatenate(aw2aw_test_stim_viterbi_decode_with_grammar_per_run),
        )
        aw2aw_stimulus_pred_letter_viterbi_decode_train_acc = (
            aw2aw_optimized_viterbi_decode_with_grammar_results[1]
        )
        aw2aw_stimulus_pred_letter_viterbi_decode_train_acc_list.append(
            aw2aw_stimulus_pred_letter_viterbi_decode_train_acc
        )
        aw2aw_stimulus_pred_letter_viterbi_decode_test_acc_list.append(
            aw2aw_stim_viterbi_decode_with_grammar_letter_accuracy
        )

        print(
            f"\taw2aw viterbi decode: train:{aw2aw_stimulus_pred_letter_viterbi_decode_train_acc:.4f} test:{aw2aw_stim_viterbi_decode_with_grammar_letter_accuracy:.4f} {param_result_format(aw2aw_optimized_viterbi_decode_with_grammar_results[0])}"
        )

        # # ===== mackenzie_soukoreff - SFFW =====
        corpus = "mackenzie_soukoreff"
        print(f"Grammar decode using corpus: {corpus}")
        prefix = ""
        symbol_nodes = grammar_info[f"{prefix}{corpus}_words_node_symbols"]
        symbol_node_edges = grammar_info[f"{prefix}{corpus}_words_link_start_end"]
        node_output_letter = grammar_info[f"unique_{corpus}_word_dictionary"]

        (
            symbol_node_trans,
            symbol_nodes_spelling_ndx,
            symbol_nodes_out_spelling_matrix,
            symbol_nodes_out_len,
            dummy_start_node_i,
            end_node_index,
        ) = symbol_info_preprocess(
            letter_label, symbol_nodes, symbol_node_edges, node_output_letter
        )

        mackenzie_soukoreff_optimized_viterbi_decode_with_grammar = partial(
            viterbi_decode_with_grammar_for_all_runs,
            dummy_start_node_i=dummy_start_node_i,
            end_node_index=end_node_index,
            symbol_node_trans=symbol_node_trans,
            symbol_nodes_spelling_ndx=symbol_nodes_spelling_ndx,
            symbol_nodes_out_spelling_matrix=symbol_nodes_out_spelling_matrix,
            symbol_nodes_out_len=symbol_nodes_out_len,
            uni_count=uni_gram_count,
            bi_count=bigram_gram_count,
            initial_proba=initial_proba,
            **optimized_viterbi_decode_with_grammar_results[0],
        )

        test_stim_viterbi_decode_with_grammar_per_run = (
            mackenzie_soukoreff_optimized_viterbi_decode_with_grammar(
                letter_proba_per_run=test_naive_letter_proba_marg_per_run
            )
        )
        stimulus_pred_letter_viterbi_decode_letter_label_list += (
            test_stim_viterbi_decode_with_grammar_per_run
        )
        stim_viterbi_decode_with_grammar_letter_accuracy = accuracy_score(
            test_letter_label_i_flatten,
            np.concatenate(test_stim_viterbi_decode_with_grammar_per_run),
        )
        mackenzie_soukoreff_pred_letter_viterbi_decode_train = (
            optimized_viterbi_decode_with_grammar_results[1]
        )
        mackenzie_soukoreff_pred_letter_viterbi_decode_train_list.append(
            mackenzie_soukoreff_pred_letter_viterbi_decode_train
        )
        mackenzie_soukoreff_pred_letter_viterbi_decode_test_list.append(
            stim_viterbi_decode_with_grammar_letter_accuracy
        )

        print(
            f"\tSFFW  viterbi decode: train:{mackenzie_soukoreff_pred_letter_viterbi_decode_train:.4f} test:{stim_viterbi_decode_with_grammar_letter_accuracy:.4f} {param_result_format(optimized_viterbi_decode_with_grammar_results[0])}"
        )

        # # ===== mackenzie_soukoreff - aw2aw =====
        prefix = "aw2aw_"
        symbol_nodes = grammar_info[f"{prefix}{corpus}_words_node_symbols"]
        symbol_node_edges = grammar_info[f"{prefix}{corpus}_words_link_start_end"]
        node_output_letter = grammar_info[f"unique_{corpus}_word_dictionary"]

        (
            symbol_node_trans,
            symbol_nodes_spelling_ndx,
            symbol_nodes_out_spelling_matrix,
            symbol_nodes_out_len,
            dummy_start_node_i,
            end_node_index,
        ) = symbol_info_preprocess(
            letter_label, symbol_nodes, symbol_node_edges, node_output_letter
        )

        aw2aw_mackenzie_soukoreff_optimized_viterbi_decode_with_grammar = partial(
            viterbi_decode_with_grammar_for_all_runs,
            dummy_start_node_i=dummy_start_node_i,
            end_node_index=end_node_index,
            symbol_node_trans=symbol_node_trans,
            symbol_nodes_spelling_ndx=symbol_nodes_spelling_ndx,
            symbol_nodes_out_spelling_matrix=symbol_nodes_out_spelling_matrix,
            symbol_nodes_out_len=symbol_nodes_out_len,
            uni_count=uni_gram_count,
            bi_count=bigram_gram_count,
            initial_proba=initial_proba,
            **aw2aw_optimized_viterbi_decode_with_grammar_results[0],
        )
        aw2aw_test_stim_viterbi_decode_with_grammar_per_run = (
            aw2aw_mackenzie_soukoreff_optimized_viterbi_decode_with_grammar(
                letter_proba_per_run=test_naive_letter_proba_marg_per_run
            )
        )
        aw2aw_stimulus_pred_letter_viterbi_decode_letter_label_list += (
            aw2aw_test_stim_viterbi_decode_with_grammar_per_run
        )
        aw2aw_stim_viterbi_decode_with_grammar_letter_accuracy = accuracy_score(
            test_letter_label_i_flatten,
            np.concatenate(aw2aw_test_stim_viterbi_decode_with_grammar_per_run),
        )
        aw2aw_mackenzie_soukoreff_pred_letter_viterbi_decode_train = (
            aw2aw_optimized_viterbi_decode_with_grammar_results[1]
        )
        aw2aw_mackenzie_soukoreff_pred_letter_viterbi_decode_train_list.append(
            aw2aw_mackenzie_soukoreff_pred_letter_viterbi_decode_train
        )
        aw2aw_mackenzie_soukoreff_pred_letter_viterbi_decode_test_list.append(
            aw2aw_stim_viterbi_decode_with_grammar_letter_accuracy
        )
        print(
            f"\taw2aw viterbi decode: train:{aw2aw_mackenzie_soukoreff_pred_letter_viterbi_decode_train:.4f} test:{aw2aw_stim_viterbi_decode_with_grammar_letter_accuracy:.4f} {param_result_format(aw2aw_optimized_viterbi_decode_with_grammar_results[0])}"
        )

        fold_i_end_time = time.time()
        print(f"{fold_i_end_time - fold_i_start_time:.4f}s")
        # break

    total_time_end_i = time.time()
    print(f"total: {total_time_end_i - total_time_start_i:.4f}s")

In [ ]:
# test_state_label_list = np.array(test_state_label_list)
region_test_state_pred_list_concat = np.concatenate(region_test_state_pred_list)
test_state_label_list_flatten = np.concatenate(test_state_label_list)
region_test_state_pred_list_flatten = np.concatenate(region_test_state_pred_list_concat)
print("----- region train test results -----")
for i, r in enumerate(regressor_types):
    print(f"region:{r:>4} acc: {accuracy_score(test_state_label_list_flatten[:,i], region_test_state_pred_list_flatten[:, i]):7.4f}")
print("----- ------------------- ----")

naive_accuracy = accuracy_score(
    [j for i in test_letter_label_list for j in i],
    [j for i in test_naive_prob_letter_pred_list for j in i],
)
print(f"naive accuracy: {naive_accuracy:.4f}")

if FILE_TO_USE not in ["6S", "18S"]:

    stim_forward_decode_accuracy = accuracy_score(
        [j for i in test_letter_label_list for j in i],
        [j for i in stimulus_forward_decode_letter_list for j in i],
    )

    stim_viterbi_decode_accuracy = accuracy_score(
        [j for i in test_letter_label_list for j in i],
        [j for i in stimulus_viterbi_decode_letter_list for j in i],
    )

    mackenzie_soukoreff_forward_decode_accuracy = accuracy_score(
        [j for i in test_letter_label_list for j in i],
        [j for i in mackenzie_soukoreff_forward_decode_letter_list for j in i],
    )

    mackenzie_soukoreff_viterbi_decode_accuracy = accuracy_score(
        [j for i in test_letter_label_list for j in i],
        [j for i in mackenzie_soukoreff_viterbi_decode_letter_list for j in i],
    )

    sffw_stim_grammar_acc = np.mean(stimulus_pred_letter_viterbi_decode_test_acc_list)
    aw2aw_stim_grammar_acc = np.mean(
        aw2aw_stimulus_pred_letter_viterbi_decode_test_acc_list
    )
    sffw_machenzie_soukoreff_grammar_acc = np.mean(
        mackenzie_soukoreff_pred_letter_viterbi_decode_test_list
    )
    aw2aw_machenzie_soukoreff_grammar_acc = np.mean(
        aw2aw_mackenzie_soukoreff_pred_letter_viterbi_decode_test_list
    )

    print("\nletter decoding:")
    print(
        f"stim\n\tforward acc:{stim_forward_decode_accuracy:.4f}\n\tviterbi acc:{stim_viterbi_decode_accuracy:.4f}"
    )
    print(
        f"mackenzie_soukoreff\n\tforward acc:{mackenzie_soukoreff_forward_decode_accuracy:.4f}\n\tviterbi acc:{mackenzie_soukoreff_viterbi_decode_accuracy:.4f}"
    )
    print("\ngrammar decoding:")
    print(
        f"stim\n\tSFFW acc:{sffw_stim_grammar_acc:.4f}\n\taw2aw acc:{aw2aw_stim_grammar_acc:.4f}"
    )
    print(
        f"mackenzie_soukoreff\n\tSFFW acc:{sffw_machenzie_soukoreff_grammar_acc:.4f}\n\taw2aw acc:{aw2aw_machenzie_soukoreff_grammar_acc:.4f}"
    )

In [ ]:
if EXPERIMENT_TYPE == "subj_dependent":
    SVM_result_cache_path = f"./brainbraille_SVM_result_cache_{EXPERIMENT_TYPE}_{FILE_TO_USE}_{STIMULI_LABEL_INTERVAL_s}s_sub_{sub}_test_size_{n}_{datetime.now(pytz.UTC).strftime('%m_%d_%H_%M_%S')}.bin"
else:
    SVM_result_cache_path = f"./brainbraille_SVM_result_cache_{EXPERIMENT_TYPE}_{FILE_TO_USE}_{STIMULI_LABEL_INTERVAL_s}s_test_sub_{test_sub}_{datetime.now(pytz.UTC).strftime('%m_%d_%H_%M_%S')}.bin"


cached_results = {
    "regressor_types": regressor_types,
    "test_letter_label_list": test_letter_label_list,
    "test_state_label_list": test_state_label_list,

    "test_naive_prob_letter_pred_list": test_naive_prob_letter_pred_list,

    "region_train_valid_state_label_list": region_train_valid_state_label_list,
    "region_train_valid_state_pred_list": region_train_valid_state_pred_list,
    "region_test_state_pred_list": region_test_state_pred_list,

    "region_cv_score_train_list": region_cv_score_train_list,
    "region_cv_score_test_list": region_cv_score_test_list,

    "test_naive_prob_letter_test_acc_list": test_naive_prob_letter_test_acc_list,

    "stimulus_forward_decode_letter_list": stimulus_forward_decode_letter_list,
    "stimulus_forward_decode_train_acc_list": stimulus_forward_decode_train_acc_list,
    "stimulus_forward_decode_test_acc_list": stimulus_forward_decode_test_acc_list,
    "stimulus_viterbi_decode_letter_list": stimulus_viterbi_decode_letter_list,
    "stimulus_viterbi_decode_train_acc_list": stimulus_viterbi_decode_train_acc_list,
    "stimulus_viterbi_decode_test_acc_list": stimulus_viterbi_decode_test_acc_list,
    "mackenzie_soukoreff_forward_decode_letter_list": mackenzie_soukoreff_forward_decode_letter_list,
    "mackenzie_soukoreff_forward_decode_train_acc_list": mackenzie_soukoreff_forward_decode_train_acc_list,
    "mackenzie_soukoreff_forward_decode_test_acc_list": mackenzie_soukoreff_forward_decode_test_acc_list,
    "mackenzie_soukoreff_viterbi_decode_letter_list": mackenzie_soukoreff_viterbi_decode_letter_list,
    "mackenzie_soukoreff_viterbi_decode_train_acc_list": mackenzie_soukoreff_viterbi_decode_train_acc_list,
    "mackenzie_soukoreff_viterbi_decode_test_acc_list": mackenzie_soukoreff_viterbi_decode_test_acc_list,
    "stimulus_pred_letter_viterbi_decode_letter_label_list": stimulus_pred_letter_viterbi_decode_letter_label_list,
    "stimulus_pred_letter_viterbi_decode_train_acc_list": stimulus_pred_letter_viterbi_decode_train_acc_list,
    "stimulus_pred_letter_viterbi_decode_test_acc_list": stimulus_pred_letter_viterbi_decode_test_acc_list,
    "aw2aw_stimulus_pred_letter_viterbi_decode_letter_label_list": aw2aw_stimulus_pred_letter_viterbi_decode_letter_label_list,
    "aw2aw_stimulus_pred_letter_viterbi_decode_train_acc_list": aw2aw_stimulus_pred_letter_viterbi_decode_train_acc_list,
    "aw2aw_stimulus_pred_letter_viterbi_decode_test_acc_list": aw2aw_stimulus_pred_letter_viterbi_decode_test_acc_list,
    "mackenzie_soukoreff_pred_letter_viterbi_decode_letter_label_list": mackenzie_soukoreff_pred_letter_viterbi_decode_letter_label_list,
    "mackenzie_soukoreff_pred_letter_viterbi_decode_train_list": mackenzie_soukoreff_pred_letter_viterbi_decode_train_list,
    "mackenzie_soukoreff_pred_letter_viterbi_decode_test_list": mackenzie_soukoreff_pred_letter_viterbi_decode_test_list,
    "aw2aw_mackenzie_soukoreff_pred_letter_viterbi_decode_letter_label_list": aw2aw_mackenzie_soukoreff_pred_letter_viterbi_decode_letter_label_list,
    "aw2aw_mackenzie_soukoreff_pred_letter_viterbi_decode_train_list": aw2aw_mackenzie_soukoreff_pred_letter_viterbi_decode_train_list,
    "aw2aw_mackenzie_soukoreff_pred_letter_viterbi_decode_test_list": aw2aw_mackenzie_soukoreff_pred_letter_viterbi_decode_test_list,
}
print(SVM_result_cache_path)
write_file(
    msgpack.packb(cached_results, use_bin_type=True), SVM_result_cache_path, "wb"
)

In [ ]:
# print(len(stimulus_forward_decode_test_acc))